In [19]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline



In [20]:
#df18to21_840cap['sentence_length'] = [840 if i >= 840 else i for i in df18to21_840cap['sentence_length']]

In [21]:
df_data = pd.read_csv('regression_data.csv')

In [22]:
X = df_data.drop(columns=['sentence_length','guideline_range','guideline_var_pct','region','white','perc_charged',
                         'Unnamed: 0','Unnamed: 0.2','Unnamed: 0.1',])
y = df_data['sentence_length']

In [23]:
numeric = ['count_convictons','age']
categorical = ['year_sentenced','dependents','race','disposition','citizen', 'state',
              'criminal_hist', 'drug_type','weapon','gender','crime_type','case_type','presentence_stat',
              'sentence_type','college']

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

ctx = ColumnTransformer(
    [('ss',StandardScaler(),numeric),
     ('ohe',OneHotEncoder(handle_unknown='ignore'),\
     categorical)],
     verbose_feature_names_out=False,
     remainder = 'passthrough'
)

ctx.fit(X_train)
X_train_t = ctx.transform(X_train)
X_test_t = ctx.transform(X_test)

X_train = pd.DataFrame(X_train_t.A,columns=ctx.get_feature_names_out())
X_test = pd.DataFrame(X_test_t.A,columns=ctx.get_feature_names_out())

In [25]:
lasso = Lasso(alpha=.001,max_iter=10000,random_state=42)
lasso.fit(X_train,y_train)
lasso.score(X_train,y_train), lasso.score(X_test,y_test)

(0.5261431286697706, 0.5364949039384008)

### Creating Coefficients DataFrames for Plotting, Analysis

In [26]:
lc_dff = pd.DataFrame()
lc_dff['feature'] = [i for i in ctx.get_feature_names_out()]
lc_dff['coeffs'] = [i for i in lasso.coef_]

In [27]:
state_coeff = lc_dff[lc_dff['feature'].str.contains("state")]
state_coeff['feature'] = [i.split('_')[1] for i in state_coeff['feature']]
state_coeff = state_coeff.rename(columns={'feature':'state'})
state_coeff.head()

/var/folders/pr/wgwfmfsx53n97rhpm9z12wqh0000gn/T/ipykernel_63195/4187077159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_coeff['feature'] = [i.split('_')[1] for i in state_coeff['feature']]


,state,coeffs
21,Alabama,3.066530
22,Alaska,-4.649757
23,Arizona,-2.076129
24,Arkansas,6.494063
25,California,-8.494445


In [28]:
crime_coeff = lc_dff[lc_dff['feature'].str.contains("crime")]
crime_coeff.head()

,feature,coeffs
86,crime_type_1,-18.098917
87,crime_type_2,-6.916896
88,crime_type_3,26.198928
89,crime_type_4,13.733142
90,crime_type_5,-3.430918
